In [16]:
## import libraries

import pandas as pd 
import nltk
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [2]:
## import pkl
df = pd.read_pickle("data/jacinda.pkl")

In [3]:
df.head(10)

,date,username,to,replies,retweets,favorites,text,mentions,hashtags,day
0,2020-05-18 11:44:09,shyleexxxx,0,0,0,0,world leaders need to follow by jacinda ardern...,0,0,2020-05-18
1,2020-05-18 10:58:23,Germans4Brexit,0,3,6,32,A FBPE Rejoiner made the point that Jacinda Ad...,0,0,2020-05-18
2,2020-05-18 10:39:37,MasterDavidP,0,0,0,0,Jacinda Ardern New Zealand PM deserves a Nobel...,0,0,2020-05-18
3,2020-05-17 14:05:19,ammarkhi,0,0,0,0,Jacinda Ardern turned away from cafe because i...,0,0,2020-05-17
4,2020-05-17 08:20:46,aayatkhn,0,0,0,1,How to successfully mitigate Covid19? act like...,0,#Corona #covid1948 #lockdown #Pakistan,2020-05-17
5,2020-05-16 22:22:56,DilonJayaweera,Nikshazz,1,0,1,Jacinda is not even in the ball park to GR who...,0,0,2020-05-16
6,2020-05-16 20:26:24,waqas_mansoor,MazharAbbasGEO,0,0,1,"call me skeptic, but mostly these kind of even...",0,0,2020-05-16
7,2020-05-16 15:23:20,Xy5Z89,0,1,1,1,#NewZeeland #Wellington #Coronavirus The New Z...,0,#NewZeeland #Wellington #Coronavirus,2020-05-16
8,2020-05-16 05:00:24,Aruna1Iyengar,0,0,0,0,Thank you Jacinda and Angela! We need more of ...,0,0,2020-05-16
9,2020-05-15 21:00:25,coreluminous,0,0,0,0,so now I have my first Corona double AA side s...,0,0,2020-05-15


In [15]:
print(df.shape)


(5967, 10)


In [4]:
## make a list of tweets
text = set(list(df["text"]))

In [5]:
## using NLTK sentiment vader and for loop to get polarity scores for each tweet

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for line in text:
    pol_score = sia.polarity_scores(line)
    pol_score['text'] = line
    results.append(pol_score)



In [6]:
## add results to dataframe
res = pd.DataFrame(results)

In [7]:
res

,neg,neu,pos,compound,text
0,0.298,0.526,0.176,-0.7184,Well said Jacinda. Take No Notice of that stup...
1,0.033,0.839,0.128,0.6249,Jacinda‚Äôs Game Plan was clear Go Hard & Go E...
2,0.000,0.896,0.104,0.5563,Highly impressive leadership by Jacinda Ardern...
3,0.155,0.845,0.000,-0.2960,Covid 19 coronavirus: Jacinda Ardern reveals $...
4,0.093,0.907,0.000,-0.6249,"That Rajasthan Sarpanch of Bhilwara, Kismat Gu..."
...,...,...,...,...,...
5084,0.081,0.837,0.082,0.0085,@mrjamesob COVID-19 basic humanity Norway‚Äôs ...
5085,0.167,0.743,0.089,-0.4756,More people have been killed by jacinda voters...
5086,0.000,1.000,0.000,0.0000,Jacinda has said the COVID economic package wi...
5087,0.183,0.756,0.062,-0.7264,Walk the Walk Jacinda - It‚Äôs a really bad lo...


In [8]:
## merge with original df
result = pd.merge(df, res, how='outer', on=['text'])

In [9]:
## sort polarity into neutral, positive and negative

neu = (result['compound'] <= -0.5) & (result['compound'] < 0.5)
result.loc[neu, 'label'] = "Neutral"
result.loc[result['compound'] > 0.5, 'label'] = "Positive"
result.loc[result['compound'] < -0.5, 'label'] = "Negative"


In [10]:
result["label"] = result["label"].fillna("Neutral")

In [11]:
## get dummies for count

pd.get_dummies(result["label"], drop_first=True)

,Neutral,Positive
0,0,0
1,1,0
2,0,1
3,0,0
4,0,1
...,...,...
5962,1,0
5963,0,0
5964,1,0
5965,0,0


In [18]:
label_count = result.groupby('label').count()['text'].reset_index().sort_values(by='text',ascending=False)
label_count.style.background_gradient(cmap='Blues')

,label,text
1,Neutral,3892
2,Positive,1367
0,Negative,708


In [19]:
fig = go.Figure(go.Funnelarea(
    text = label_count.label,
    values = label_count.text,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()

In [14]:
print("Positive headlines:\n")
print(list(result[result['label'] == "Positive"].text)[:2])


print("Neutral headlines:\n")
print(list(result[result['label'] == "Neutral"].text)[:2])


print("\nNegative headlines:\n")
print(list(result[result['label'] == "Negative"].text)[:2])

Positive headlines:

['Jacinda Ardern New Zealand PM deserves a Nobel Prize for how she handled corona pandemic and Christ church massacre.', 'How to successfully mitigate Covid19? act like you have Covid19 for 14 days. Jacinda Ardern, New Zealand PM #Corona #covid1948 #lockdown #Pakistan']
Neutral headlines:

['A FBPE Rejoiner made the point that Jacinda Adern, Katrin Jakobsdottir and Erna Solberg handled the Corona crisis very well compared to the UK government. I just made the point that Icelandic PM Jakobsdottir is anti-EU. 1 second later, the FBPE guy blocke', 'Jacinda is not even in the ball park to GR who has single-handed eradicated Corona like he did with LTTE. The world must put him on a pedestal and worship him as they do in Sri Lanka. Curfew has lifted but still counting 980 odd cases. Temples closed but ']

Negative headlines:

['world leaders need to follow by jacinda ardern when it comes to responding to sudden crises , (like her gun ban and corona virus outbreak) queen'

In [ ]:
## From reading 